<a href="https://colab.research.google.com/github/janali-17/RAG_Project/blob/main/RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install langchain langchain-community openai tiktoken pinecone-client langchain_pinecone unstructured pdfminer==20191125 pdfminer.six==20221105 pillow_heif unstructured_inference youtube-transcript-api pytube sentence-transformers



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 29.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 26.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 7.7 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of pdfplumber to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 46.3 MB/s eta 0:00:00
   ━━━━━━

In [ ]:
from langchain.document_loaders import UnstructuredPDFLoader, OnlinePDFLoader, WebBaseLoader, YoutubeLoader, DirectoryLoader, TextLoader, PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from sklearn.metrics.pairwise import cosine_similarity
from langchain_pinecone import PineconeVectorStore
from langchain.embeddings import OpenAIEmbeddings
from langchain_community.embeddings import HuggingFaceEmbeddings
from google.colab import userdata
from pinecone import Pinecone, ServerlessSpec
from openai import OpenAI
import numpy as np
import tiktoken
import os
from langchain_community.document_loaders import YoutubeLoader

In [ ]:
pinecone_api_key = userdata.get("PINECONE_API_KEY")
os.environ['PINECONE_API_KEY'] = pinecone_api_key

In [ ]:
openrouter_client = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key=userdata.get("OPENROUTER_API")
)

In [ ]:

tokenizer = tiktoken.get_encoding('p50k_base')

def tiktoken_len(text):
    tokens = tokenizer.encode(
        text,
        disallowed_special=()
    )
    return len(tokens)

text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=2000,
        chunk_overlap=100,
        length_function=tiktoken_len,
        separators=["\n\n", "\n", " ", ""]
)

In [ ]:
hf_embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

def get_embedding(text):
    return hf_embeddings.embed_query(text)

def cosine_similarity_between_sentences(sentence1, sentence2):
    embedding1 = np.array(get_embedding(sentence1))
    embedding2 = np.array(get_embedding(sentence2))

    if embedding1 is None or embedding2 is None:
        return None

    embedding1 = embedding1.reshape(1, -1)
    embedding2 = embedding2.reshape(1, -1)

    print("Sentence 1 Embedding: ", embedding1)
    print("Sentence 2 Embedding: ", embedding2)

    similarity = cosine_similarity(embedding1, embedding2)
    return similarity[0][0]

sentence1 = "Great to finally meet!"
sentence2 = "Nice to meet you"

similarity = cosine_similarity_between_sentences(sentence1, sentence2)
if similarity is not None:
    print(f"Cosine similarity between sentences:\n'{sentence1}' and '{sentence2}': {similarity:.4f}")
else:
    print("Could not compute similarity due to an error.")

<ipython-input-6-8fa2e95f6ca8>:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  hf_embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Sentence 1 Embedding:  [[-6.42226040e-02 -2.30498277e-02  4.12189811e-02  5.67926466e-02
  -1.63038597e-02  4.15017158e-02  8.88479128e-03 -4.60831709e-02
  -1.57466414e-03 -2.54955254e-02 -3.06746550e-03  9.07608494e-03
   1.05423806e-02  3.58030386e-02  3.46321873e-02  9.65092238e-03
  -2.56988853e-02 -3.51948030e-02  3.15498598e-02  3.47745456e-02
  -5.27179986e-02 -2.66816318e-02  2.66268365e-02 -1.34273320e-02
  -5.38467579e-02  3.75799456e-04 -2.99408399e-02  4.46607508e-02
   4.49724793e-02  1.95614807e-02 -1.07248917e-01  1.15735307e-01
  -4.73707877e-02  5.04956283e-02  4.24734168e-02  3.78863029e-02
   3.02728396e-02 -3.62443663e-02  7.66253024e-02  1.54855186e-02
  -3.44800763e-02  1.85779128e-02  5.72161041e-02  5.62272444e-02
  -3.45480926e-02  4.13312130e-02  5.84770320e-03  4.52666581e-02
   1.93616562e-02  2.63053197e-02  2.68412270e-02  1.28470203e-02
   3.65549289e-02 -4.26418483e-02  2.21714508e-02  1.24915272e-01
  -1.69741251e-02 -1.28599599e-01 -3.78995319e-03 -1.

In [ ]:

loader = YoutubeLoader.from_youtube_url("https://www.youtube.com/watch?v=OuZrhykVytg&t=334s", add_video_info=False)
data = loader.load()
print(data)


[Document(metadata={'source': 'OuZrhykVytg'}, page_content="in this video we're going to go over what are events in c-sharp how they work and how they're used we're going to see the benefits of events and how they are essential to keeping our code nice and clean but Vienna [Music] hello and welcome I'm your code monkey and this channel is all about helping you learn how to make your own games with enough tutorials made by a professional indie game developer so if you find the video helpful consider subscribing okay so if you've seen a bunch of my videos you've probably heard me mention two things one I always highlight the importance of writing clean code and two I use events to d-cup on various systems and keep my code clean if you want to write good quality code you need to learn and use events the good thing is they're actually very easy so events aren't just a way of saying something happened without knowing or caring about who is listening if anyone you have publishers and subscri

In [ ]:
texts = text_splitter.split_documents(data)


In [ ]:
texts

[Document(metadata={'source': 'OuZrhykVytg'}, page_content="in this video we're going to go over what are events in c-sharp how they work and how they're used we're going to see the benefits of events and how they are essential to keeping our code nice and clean but Vienna [Music] hello and welcome I'm your code monkey and this channel is all about helping you learn how to make your own games with enough tutorials made by a professional indie game developer so if you find the video helpful consider subscribing okay so if you've seen a bunch of my videos you've probably heard me mention two things one I always highlight the importance of writing clean code and two I use events to d-cup on various systems and keep my code clean if you want to write good quality code you need to learn and use events the good thing is they're actually very easy so events aren't just a way of saying something happened without knowing or caring about who is listening if anyone you have publishers and subscri

In [ ]:
pinecone_api_key = userdata.get("PINECONE_API_KEY")
os.environ['PINECONE_API_KEY'] = pinecone_api_key
pc = Pinecone()
pc.create_index(
    name="projectrag", dimension=384, metric="cosine", spec=ServerlessSpec(cloud="aws", region="us-east-1")
)

PineconeApiException: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2024-07', 'X-Cloud-Trace-Context': '9dc6ab9cd7f70487db91788ca32c1a58', 'Date': 'Sat, 21 Dec 2024 13:57:19 GMT', 'Server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"ALREADY_EXISTS","message":"Resource  already exists"},"status":409}


In [ ]:
vectorstore = PineconeVectorStore(index_name="projectrag", embedding=hf_embeddings)

index_name = "projectrag"

namespace = "youtube-video"

In [ ]:
for document in texts:
  print("-----------")
  print(document.metadata, document.page_content)

-----------
{'source': 'OuZrhykVytg'} in this video we're going to go over what are events in c-sharp how they work and how they're used we're going to see the benefits of events and how they are essential to keeping our code nice and clean but Vienna [Music] hello and welcome I'm your code monkey and this channel is all about helping you learn how to make your own games with enough tutorials made by a professional indie game developer so if you find the video helpful consider subscribing okay so if you've seen a bunch of my videos you've probably heard me mention two things one I always highlight the importance of writing clean code and two I use events to d-cup on various systems and keep my code clean if you want to write good quality code you need to learn and use events the good thing is they're actually very easy so events aren't just a way of saying something happened without knowing or caring about who is listening if anyone you have publishers and subscribers the publisher has

In [ ]:
vectorstore = PineconeVectorStore(index_name=index_name, embedding=hf_embeddings)

In [ ]:
precomputed_embeddings = [
    hf_embeddings.embed_query(
        f"Source: {t.metadata.get('source', 'Unknown Source')}, "
        f"Title: {t.metadata.get('title', 'Untitled')} \n\n"
        f"Content: {t.page_content}"
    )
    for t in texts
]

In [ ]:
for i, document in enumerate(texts):
    embedding = precomputed_embeddings[i]
    vectorstore.add_texts(
        texts=[document.page_content],
        metadatas=[document.metadata],
        embeddings=[embedding],
        namespace="youtube-video"
    )

In [ ]:
pc = Pinecone(api_key=userdata.get("PINECONE_API_KEY"))
pinecone_index = pc.Index("projectrag")

In [ ]:
def perform_youtube_rag(query):
    query_embedding = hf_embeddings.embed_query(query)

    top_matches = pinecone_index.query(vector=query_embedding, top_k=10, include_metadata=True, namespace=namespace)

    contexts = [item['metadata']['text'] for item in top_matches['matches']]

    augmented_query = "\n" + "\n\n-------\n\n".join(contexts[ : 10]) + "\n-------\n\n\n\n\nMY QUESTION:\n" + query

    system_prompt = f"""You are an expert personal assistant. Answer any questions I have about the Youtube Video provided. You always answer questions based only on the context that you have been provided.
    """

    res = openrouter_client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": augmented_query}
    ]
    )

    return res.choices[0].message.content

In [ ]:
perform_youtube_rag("How to Create Events in Unity")

'To create events in Unity, you can follow these general steps based on the provided context from the video:\n\n1. Define an Event: Start by defining an event within your script. You can create an event by using the `event` keyword, specifying the access modifier (such as public), and assigning a delegate type. The delegate type commonly used for events in C# is `EventHandler`.\n\n2. Trigger the Event: To trigger the event, you can call the event as if it were a function. This is typically done when a specific action or condition occurs in your game.\n\n3. Subscribe to the Event: Define a function that will receive the event when it is triggered. The function signature should match the delegate type of the event. Subscribe to the event by using the `+=` operator and adding your function to the list of event subscribers.\n\n4. Unsubscribe from the Event: If needed, you can unsubscribe from the event by using the `-=` operator, removing your function from the list of event subscribers.\n

In [ ]:
loader = PyPDFLoader("Statistics-book.pdf")
data = loader.load()

print(data)

text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=2000,
        chunk_overlap=100,
        length_function=tiktoken_len,
        separators=["\n\n", "\n", " ", ""]
    )

texts = text_splitter.split_documents(data)

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

vectorstore = PineconeVectorStore(index_name=index_name, embedding=embeddings)

vectorstore_from_texts = PineconeVectorStore.from_texts(
    [f"Source: {t.metadata.get('source', 'Unknown')}, Title: {t.metadata.get('title', 'Unknown')} \n\nContent: {t.page_content}" for t in texts],
    embeddings,
    index_name=index_name,
    namespace=namespace
)

[Document(metadata={'source': 'Statistics-book.pdf', 'page': 0}, page_content='STA301 – Statistics and Probability                                                                                              \n \nVirtual University of Pakistan                                                                                                       i                                                                                                                                           \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \nVirtual University of Pakistan \n \nStatistics and Probability \n \nSTA301 '), Document(metadata={'source': 'Statistics-book.pdf', 'page': 1}, page_content='STA301 – Statistics and Probability                                                                                              \n \nVirtual University of Pakistan                                                                                                       ii                             

In [ ]:
def perform_pdf_rag(query):
    query_embedding = hf_embeddings.embed_query(query)

    top_matches = pinecone_index.query(vector=query_embedding, top_k=10, include_metadata=True, namespace=namespace)

    contexts = [item['metadata']['text'] for item in top_matches['matches']]

    augmented_query = "\n" + "\n\n-------\n\n".join(contexts[ : 10]) + "\n-------\n\n\n\n\nMY QUESTION:\n" + query

    primer = "You are a personal assistant. Answer any questions I have about the PDF file provided."

    system_prompt = f"""You are an expert personal assistant. Answer any questions I have about the PDF file provided. You always answer questions based only on the context that you have been provided.
    """

    res = openrouter_client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": primer},
        {"role": "user", "content": augmented_query}
    ]
    )

    return res.choices[0].message.content

In [ ]:
perform_pdf_rag("what is nominal scale?")

'A nominal scale is a classification or grouping of observations into distinct, qualitative categories or classes. It involves assigning numbers or labels to categories without any inherent order or numerical significance. For example, classifying students as male or female, or rainfall as heavy, moderate, or light are examples of nominal scales. The numbers used for identification in a nominal scale do not imply any specific order or quantity.'